This notebook creates and trains a FCN model which deals with count-based encoded data.

In [1]:
# Read data

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Possible critical alarm types
critical_alarm_types = [7,15,16,21,33,56,68,95,1000,1001]
df = pd.DataFrame()

for i in critical_alarm_types:
    # Replace the link according to the data you want to read (All, Unique Samples, Random Samples)
    type_df = pd.read_csv("../Data/Final Data/Train/Random Samples/Count-based/" + str(i) + "_countbased.csv")
    df = pd.concat([df, type_df], ignore_index=True)

X = df.drop(columns=['y']).values
y = df['y'].values

If you wish to perform an upsampling using SMOTE, run the following cell. If you do not wish to perform an upsampling, bypass the following cell and run the next one.

In [ ]:
# Upsampling using SMOTE
from imblearn.over_sampling import SMOTE

# Apply SMOTE to balance the dataset
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X, y)

In [ ]:
# 1-hot-encoding of the labels and train-test-split

# Determine the number of unique critical error types
num_classes = len(np.unique(y))

# Encode the target variable using LabelEncoder and one-hot encoding
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = to_categorical(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
# Creating the architecture of the FCN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, GlobalAveragePooling1D, Dense, BatchNormalization, Dropout

def create_model(filters, kernel_size):
    # Create the FCN model
    model = Sequential()

    # Encoder
    model.add(Conv1D(filters, kernel_size, activation='relu', padding='same', input_shape=(77, 1)))
    model.add(Conv1D(filters, kernel_size, activation='relu', padding='same'))
    model.add(Conv1D(filters*2, kernel_size, activation='relu', padding='same'))
    model.add(Conv1D(filters*2, kernel_size, activation='relu', padding='same'))

    # Decoder
    model.add(Conv1D(filters, kernel_size, activation='relu', padding='same'))
    model.add(Conv1D(filters, kernel_size, activation='relu', padding='same'))

    # Global average pooling
    model.add(GlobalAveragePooling1D())

    # Dense layer for classification
    model.add(Dense(10, activation='softmax'))
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Print the model summary

model = create_model(64,9)

In [ ]:
from keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor="val_loss",
                                        mode="min", patience=3,
                                        restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, batch_size=64, epochs=50, validation_data=(X_test, y_test), callbacks=[earlystopping])

In [ ]:
# Save model
import joblib

joblib.dump(model, 'FCN_countbased.joblib')

In [ ]:
# Prediction with test data

y_pred = model.predict(X_test)

In [ ]:
# Undo 1-hot-encoding of the class labels

import numpy as np

y_pred_classes = []
y_real_classes = []

# Undo 1-hot-encoding of labels
for item in y_pred:
    y_pred_classes.append(np.argmax(item))

for item in y_test:
    y_real_classes.append(np.argmax(item))

In [ ]:
# Classification report for train-test-split
from sklearn.metrics import classification_report

label_names = ['7', '15', '16', '21', '33', '56', '68', '95', '1000', '1001']
print(classification_report(y_real_classes, y_pred_classes, target_names=label_names))

In [ ]:
# Print confusion matrix

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

label_names = ['7', '15', '16', '21', '33', '56', '68', '95', '1000', '1001']

cm = confusion_matrix(y_real_classes, y_pred_classes)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=label_names)
disp.plot()
plt.show()